In [1]:
# In this notebook, we will calculate the change in oceanic heat flux convergence

# By: Ty Janoski
# edited: 06.02.20

In [2]:
# import statements
import xarray as xr
import numpy as np
import dask.array as da
import xesmf as xe
from dask.diagnostics import ProgressBar
import glob

In [20]:
# List to store the climatology
# excluded models: bcc-csm1-1, GFDL-ESM2G, IPSL-CM5B-LR, FGOALS-s2, CanESM2, inmcm4, IPSL-CM5A-LR

models = ['MIROC-ESM']
for mod in models:
    hold = []
    print(mod)
    for i in range(5):
        print(i)
        start = -600 + (i*120)
        end = start+120
        if(i==4):
            end = None
        if mod in ['ACCESS1-0','ACCESS1-3','CNRM-CM5']:
            # read in mixed-layer depth, as calculated in mixed_layer.ipynb
            mlotst = xr.open_dataset('/dx07/tylerj/CMIP5_output/CMIP5_feedbacks/'+mod+'_mlotst_pic.nc',
                                 use_cftime=True).mlotst.isel(time=slice(start,end)).load()
            # read potential temperature from last 50 years of piControl
            thetao = xr.open_mfdataset('/dx07/tylerj/CMIP5_output/piControl/thetao_Omon_'+mod+'_*.nc',
                                      parallel=True,combine='by_coords',
                                  use_cftime=True).thetao.isel(time=slice(start,end))
            if(i==4):
                hfds = xr.open_mfdataset('/dx07/tylerj/CMIP5_output/piControl/hfds_Omon_'+mod+'_*.nc',
                                        parallel=True,combine='by_coords',
                                        use_cftime=True).hfds.isel(time=slice(-600,None))
        elif mod in ['MIROC-ESM']:
            # read in mixed-layer depth, as calculated in mixed_layer.ipynb
            mlotst = xr.open_dataset('/dx07/tylerj/CMIP5_output/CMIP5_feedbacks/'+mod+'_mlotst_pic.nc',
                                 use_cftime=True).mlotst.isel(time=slice(start,end)).load()
            if(i==4):
                end=0
            print(start-1188,end-1188)
            # read potential temperature from last 50 years of piControl
            thetao = xr.open_mfdataset('/dx07/tylerj/CMIP5_output/piControl/thetao_Omon_'+mod+'_*.nc',
                                      parallel=True,combine='by_coords',
                                  use_cftime=True).thetao.isel(time=slice(start-1188,end-1188))
            if(i==4):
                hfds = xr.open_mfdataset('/dx07/tylerj/CMIP5_output/piControl/hfds_Omon_'+mod+'_*.nc',
                                        parallel=True,combine='by_coords',
                                        use_cftime=True).hfds.isel(time=slice(-600-1188,0-1188))
            mlotst['time'] = thetao.time
        # FGOALS-g2 time axis isn't formatted correctly, so we can't decode the time axis
        elif mod in ['FGOALS-g2']:
            # read in mixed-layer depth, as calculated in mixed_layer.ipynb
            mlotst = xr.open_dataset('/dx07/tylerj/CMIP5_output/CMIP5_feedbacks/'+mod+'_mlotst_pic.nc',
                                 decode_times=False).mlotst.isel(time=slice(start,end))
            # read potential temperature from last 50 years of piControl
            thetao = xr.open_mfdataset('/dx07/tylerj/CMIP5_output/piControl/thetao_Omon_'+mod+'_*.nc',
                                      parallel=True,combine='by_coords',
                                  decode_times=False).thetao.isel(time=slice(start,end))
            if(i==4):
                hfds = xr.open_mfdataset('/dx07/tylerj/CMIP5_output/piControl/hfds_Omon_'+mod+'_*.nc',
                                          parallel=True,combine='by_coords',
                                      decode_times=False).hfds.isel(time=slice(-600,None))
        else:
            # read in mixed-layer depth, as calculated in mixed_layer.ipynb
            mlotst = xr.open_dataset('/dx07/tylerj/CMIP5_output/CMIP5_feedbacks/'+mod+'_mlotst_pic.nc',
                                 use_cftime=True).mlotst.isel(time=slice(start,end))
            path = glob.glob('/dx07/tylerj/CMIP5_output/piControl/thetao_Omon_'+mod+'*.nc')[0]
            thetao = xr.open_dataset(path,use_cftime=True).thetao.isel(
                time=slice(start,end))
            if(i==4):
                path = glob.glob('/dx07/tylerj/CMIP5_output/piControl/hfds_Omon_'+mod+'*.nc')[0]
                hfds = xr.open_dataset(path,use_cftime=True).hfds.isel(
                    time=slice(-600,None))


        # Next: establish which points are actually includes in the previously-defined mixed-layer
        mlotst_4D = xr.broadcast(mlotst,thetao)[0]
        ml = (mlotst_4D.lev <= mlotst_4D).load()
        del mlotst_4D
        
        start = -600 + (i*120)
        end = start+120
        if(i==4):
            end = None

        if mod in ['ACCESS1-0','ACCESS1-3','GFDL-CM3','MIROC5','MPI-ESM-LR','MPI-ESM-P','MIROC-ESM']:
            # read in thkcello
            thkcello = xr.open_mfdataset('/dx07/tylerj/CMIP5_output/piControl/thkcello_Omon_'+mod+'_*.nc',
                                use_cftime=True, combine='by_coords').thkcello.isel(time=slice(start,end)).compute()
            thkcello = thkcello.where(ml)
            del ml
            weights = thkcello / thkcello.sum(dim='lev')
            del thkcello


        elif mod in ['BNU-ESM','CCSM4','FGOALS-s2','MRI-CGCM3','FGOALS-g2']:
            volcello = xr.open_mfdataset('/dx07/tylerj/CMIP5_output/piControl/volcello_fx_'+mod+'*.nc',
                                         use_cftime=True,combine='by_coords')['volcello'].load()
            areacello = xr.open_mfdataset('/dx07/tylerj/CMIP5_output/piControl/areacello_fx_'+mod+'*.nc',
                                          use_cftime=True,combine='by_coords')['areacello'].load()

            thkcello = volcello/areacello
            thkcello = xr.broadcast(thkcello,thetao)[0].where(ml)
            del ml
            weights = thkcello / thkcello.sum(dim='lev')
            del thkcello

        elif mod in ['CNRM-CM5']:
            thkcello = xr.open_dataset('/dx07/tylerj/CMIP5_output/piControl/thkcello_fx_CNRM-CM5_piControl_r0i0p0.nc').thkcello.load()
            thkcello = xr.broadcast(thkcello,thetao)[0].where(ml)
            del ml
            weights = thkcello / thkcello.sum(dim='lev')
            del thkcello

        elif mod in ['NorESM1-M']:
            # NorESM1-M is a special case where they forgot to multiply the cell thickness by areacello to get volcello
            # Therefore, volcello actually is thkcello

            thkcello = xr.open_mfdataset('/dx07/tylerj/CMIP5_output/piControl/volcello_fx_'+mod+'*.nc',
                                         use_cftime=True,combine='by_coords')['volcello'].load()

            # We need to make weights so we can take the vertical average later
            thkcello = xr.broadcast(thkcello,thetao)[0].where(ml)
            del ml
            weights = thkcello / thkcello.sum(dim='lev')
            del thkcello

        elif mod in ['CSIRO-Mk3-6-0']:
            # CSIRO-Mk3-6-0 has a climatological thkcello. We must tile it to match the length of thetao.

            thkcello = xr.open_mfdataset('/dx07/tylerj/CMIP5_output/piControl/thkcello_Omon_'+mod+'*.nc',
                                       decode_times=False,combine='by_coords')['thkcello'].load()

            # We need to make weights so we can take the vertical average later
            thkcello_with_time = xr.concat([thkcello for i in range(int(len(thetao.time)/12))],dim='time')
            thkcello_with_time['time'] = thetao['time']
            thkcello = thkcello_with_time.where(ml)
            del ml
            weights = thkcello / thkcello.sum(dim='lev')
            del thkcello
        
        with ProgressBar():
            T_ml = (thetao * weights).sum(dim='lev').compute()
        del thetao
        T_ml = T_ml.where(T_ml!=0)
        hold.append(T_ml)
    T_ml = xr.concat(hold,dim='time')
    # read in mixed-layer depth, as calculated in mixed_layer.ipynb
    mlotst = xr.open_dataset('/dx07/tylerj/CMIP5_output/CMIP5_feedbacks/'+mod+'_mlotst_pic.nc',
                             decode_times=False).mlotst.isel(time=slice(-600,None))
    if mod in ['FGOALS-g2']:
        T_ml['time'] = T_ml.time*86400
        dMLHC = T_ml.differentiate(coord='time',edge_order=2) * 3895 * np.array(mlotst) * 1025
        hfds['time'] = dMLHC.time
    else:
        dMLHC = T_ml.differentiate(coord='time',edge_order=2,datetime_unit='s') * 3895 * np.array(mlotst) * 1025
    with ProgressBar():
        if(mod!='GFDL-CM3'):
            Fconv = (dMLHC - hfds).compute()
        else:
            Fconv = (dMLHC - np.array(hfds)).compute()
    if mod in ['FGOALS-g2']:
        Fconv['time'] = np.arange(0,600,1)
        Fconv['month'] = Fconv.time%12
        clim = Fconv.groupby('month').mean(dim='time')
    else:
        clim = Fconv.groupby(Fconv.time.dt.month).mean(dim='time')

###################################################################################
################# abrupt4xCO2 below this line #################################### 
###################################################################################
    hold = []
    print(mod)
    for i in range(15):
        print(i)
        start = i * 120
        end = start+120
        if mod in ['ACCESS1-0','ACCESS1-3','CNRM-CM5','MIROC-ESM']:
            # read in mixed-layer depth, as calculated in mixed_layer.ipynb
            mlotst = xr.open_dataset('/dx07/tylerj/CMIP5_output/CMIP5_feedbacks/'+mod+'_mlotst_4xCO2.nc',
                                 use_cftime=True).mlotst.isel(time=slice(start,end)).load()
            # read potential temperature from last 50 years of piControl
            thetao = xr.open_mfdataset('/dx07/tylerj/CMIP5_output/abrupt4xCO2/thetao_Omon_'+mod+'_*.nc',
                                      parallel=True,combine='by_coords',
                                  use_cftime=True).thetao.isel(time=slice(start,end))
            if(i==14):
                hfds = xr.open_mfdataset('/dx07/tylerj/CMIP5_output/abrupt4xCO2/hfds_Omon_'+mod+'_*.nc',
                                        parallel=True,combine='by_coords',
                                        use_cftime=True).hfds.isel(time=slice(None,1800))
        # FGOALS-g2 time axis isn't formatted correctly, so we can't decode the time axis
        elif mod in ['FGOALS-g2']:
            # read in mixed-layer depth, as calculated in mixed_layer.ipynb
            mlotst = xr.open_dataset('/dx07/tylerj/CMIP5_output/CMIP5_feedbacks/'+mod+'_mlotst_4xCO2.nc',
                                 decode_times=False).mlotst.isel(time=slice(start,end))
            # read potential temperature from last 50 years of piControl
            thetao = xr.open_mfdataset('/dx07/tylerj/CMIP5_output/abrupt4xCO2/thetao_Omon_'+mod+'_*.nc',
                                      parallel=True,combine='by_coords',
                                  decode_times=False).thetao.isel(time=slice(start,end))
            if(i==14):
                hfds = xr.open_mfdataset('/dx07/tylerj/CMIP5_output/abrupt4xCO2/hfds_Omon_'+mod+'_*.nc',
                                          parallel=True,combine='by_coords',
                                      decode_times=False).hfds.isel(time=slice(None,1800))
        else:
            # read in mixed-layer depth, as calculated in mixed_layer.ipynb
            mlotst = xr.open_dataset('/dx07/tylerj/CMIP5_output/CMIP5_feedbacks/'+mod+'_mlotst_4xCO2.nc',
                                 use_cftime=True).mlotst.isel(time=slice(start,end))
            path = glob.glob('/dx07/tylerj/CMIP5_output/abrupt4xCO2/thetao_Omon_'+mod+'*.nc')[0]
            thetao = xr.open_dataset(path,use_cftime=True).thetao.isel(
                time=slice(start,end))
            if(i==14):
                path = glob.glob('/dx07/tylerj/CMIP5_output/abrupt4xCO2/hfds_Omon_'+mod+'*.nc')[0]
                hfds = xr.open_dataset(path,use_cftime=True).hfds.isel(
                    time=slice(None,1800))


        # Next: establish which points are actually includes in the previously-defined mixed-layer
        mlotst_4D = xr.broadcast(mlotst,thetao)[0]
        ml = (mlotst_4D.lev <= mlotst_4D).load()
        del mlotst_4D

        if mod in ['ACCESS1-0','ACCESS1-3','GFDL-CM3','MIROC5','MPI-ESM-LR','MPI-ESM-P','MIROC-ESM']:
            # read in thkcello
            thkcello = xr.open_mfdataset('/dx07/tylerj/CMIP5_output/abrupt4xCO2/thkcello_Omon_'+mod+'_*.nc',
                                use_cftime=True, combine='by_coords').thkcello.isel(time=slice(start,end)).compute()
            thkcello = thkcello.where(ml)
            del ml
            weights = thkcello / thkcello.sum(dim='lev')
            del thkcello
            
        elif mod in ['CNRM-CM5']:
            thkcello = xr.open_dataset('/dx07/tylerj/CMIP5_output/piControl/thkcello_fx_CNRM-CM5_piControl_r0i0p0.nc').thkcello.load()
            thkcello = xr.broadcast(thkcello,thetao)[0].where(ml)
            del ml
            weights = thkcello / thkcello.sum(dim='lev')
            del thkcello

        elif mod in ['BNU-ESM','CCSM4','FGOALS-s2','MRI-CGCM3','FGOALS-g2']:
            volcello = xr.open_mfdataset('/dx07/tylerj/CMIP5_output/piControl/volcello_fx_'+mod+'*.nc',
                                         use_cftime=True,combine='by_coords')['volcello'].load()
            areacello = xr.open_mfdataset('/dx07/tylerj/CMIP5_output/piControl/areacello_fx_'+mod+'*.nc',
                                          use_cftime=True,combine='by_coords')['areacello'].load()

            thkcello = volcello/areacello
            thkcello = xr.broadcast(thkcello,thetao)[0].where(ml)
            del ml
            weights = thkcello / thkcello.sum(dim='lev')
            del thkcello


        elif mod in ['NorESM1-M']:
            # NorESM1-M is a special case where they forgot to multiply the cell thickness by areacello to get volcello
            # Therefore, volcello actually is thkcello

            thkcello = xr.open_mfdataset('/dx07/tylerj/CMIP5_output/piControl/volcello_fx_'+mod+'*.nc',
                                         use_cftime=True,combine='by_coords')['volcello'].load()

            # We need to make weights so we can take the vertical average later
            thkcello = xr.broadcast(thkcello,thetao)[0].where(ml)
            del ml
            weights = thkcello / thkcello.sum(dim='lev')
            del thkcello

        elif mod in ['CSIRO-Mk3-6-0']:
            # CSIRO-Mk3-6-0 has a climatological thkcello. We must tile it to match the length of thetao.

            thkcello = xr.open_mfdataset('/dx07/tylerj/CMIP5_output/piControl/thkcello_Omon_'+mod+'*.nc',
                                       decode_times=False,combine='by_coords')['thkcello'].load()

            # We need to make weights so we can take the vertical average later
            thkcello_with_time = xr.concat([thkcello for i in range(int(len(thetao.time)/12))],dim='time')
            thkcello_with_time['time'] = thetao['time']
            thkcello = thkcello_with_time.where(ml)
            del ml
            weights = thkcello / thkcello.sum(dim='lev')
            del thkcello
        
        with ProgressBar():
            T_ml = (thetao * weights).sum(dim='lev').compute()
        del thetao
        T_ml = T_ml.where(T_ml!=0)
        hold.append(T_ml)
    T_ml = xr.concat(hold,dim='time')
    # read in mixed-layer depth, as calculated in mixed_layer.ipynb
    mlotst = xr.open_dataset('/dx07/tylerj/CMIP5_output/CMIP5_feedbacks/'+mod+'_mlotst_4xCO2.nc',
                             decode_times=False).mlotst.isel(time=slice(None,1800))
    if mod in ['FGOALS-g2']:
        T_ml['time'] = T_ml.time*86400
        dMLHC = T_ml.differentiate(coord='time',edge_order=2) * 3895 * np.array(mlotst) * 1025
        hfds['time'] = dMLHC.time
    else:
        dMLHC = T_ml.differentiate(coord='time',edge_order=2,datetime_unit='s') * 3895 * np.array(mlotst) * 1025
    if mod in ['CCSM4']:
        hfds['time'] = dMLHC.time
    with ProgressBar():
        if(mod!='GFDL-CM3'):
            Fconv = (dMLHC - hfds).compute()
        else:
            Fconv = (dMLHC - np.array(hfds)).compute()
    diff = Fconv - np.tile(clim,(150,1,1))
    diff.to_netcdf('/dx07/tylerj/CMIP5_output/CMIP5_feedbacks/'+mod+'_ocean_conv.nc')

MIROC-ESM
0
-1788 -1668
[########################################] | 100% Completed |  2.5s
1
-1668 -1548
[########################################] | 100% Completed |  2.4s
2
-1548 -1428
[########################################] | 100% Completed |  2.4s
3
-1428 -1308
[########################################] | 100% Completed |  2.5s
4
-1308 -1188
[########################################] | 100% Completed |  7.5s
[########################################] | 100% Completed |  0.9s


/home/tylerj/miniconda3/envs/xesmf_env/lib/python3.7/site-packages/xarray/core/nanops.py:142: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis=axis, dtype=dtype)


MIROC-ESM
0
[########################################] | 100% Completed |  2.5s
1
[########################################] | 100% Completed |  2.6s
2
[########################################] | 100% Completed |  2.6s
3
[########################################] | 100% Completed |  2.6s
4
[########################################] | 100% Completed |  2.9s
5
[########################################] | 100% Completed |  3.9s
6
[########################################] | 100% Completed |  3.3s
7
[########################################] | 100% Completed |  2.8s
8
[########################################] | 100% Completed |  2.8s
9
[########################################] | 100% Completed |  2.8s
10
[########################################] | 100% Completed |  2.8s
11
[########################################] | 100% Completed |  3.0s
12
[########################################] | 100% Completed |  3.4s
13
[########################################] | 100% Completed |  3.3s
14
[##

In [19]:
weights

<xarray.DataArray 'thkcello' (time: 0, lev: 44, lat: 192, lon: 256)>
array([], shape=(0, 44, 192, 256), dtype=float32)
Coordinates:
  * time     (time) object 
  * lev      (lev) float64 2.5 7.5 12.5 17.5 ... 5.025e+03 5.275e+03 5.45e+03
  * lon      (lon) float64 0.7031 2.109 3.516 4.922 ... 355.1 356.5 357.9 359.3
  * lat      (lat) float64 -89.14 -87.43 -85.87 -84.46 ... 85.87 87.43 89.14